# Strategy Research: Beta-Neutral Short Premium

Research notebook for developing and testing the beta-neutral short premium strategy.

**Strategy Overview:**
- Core: Short strangles/iron condors on single-name equities
- Hedge: Beta-weighted delta neutral using SPX/ES
- Goal: Harvest variance risk premium while eliminating market direction risk

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from src.options_db import OptionsDB
from src.market_data import MarketData

plt.style.use('seaborn-v0_8-whitegrid')
print('Setup complete')

In [ ]:
# Connect
db = OptionsDB()
md = MarketData()

## Strategy Parameters

Define the key parameters for the strategy.

In [ ]:
# Strategy parameters
PARAMS = {
    # Position construction
    'structure': 'strangle',      # 'strangle' or 'iron_condor'
    'dte_target': 45,             # Days to expiration
    'dte_min': 30,                # Minimum DTE to open
    'dte_max': 60,                # Maximum DTE to open
    'delta_target': 0.16,         # Target delta for wings
    'delta_tolerance': 0.05,      # Acceptable delta range
    
    # Risk management
    'profit_take_pct': 0.50,      # Close at 50% profit
    'loss_limit_mult': 2.0,       # Close if loss > 2x credit
    'roll_dte': 21,               # Roll when DTE < this
    
    # Beta hedging
    'beta_lookback': 60,          # Days for beta calculation
    'hedge_threshold': 0.002,     # Re-hedge when BWD > 0.2% of portfolio
    
    # Position sizing
    'max_vega_per_name': 0.08,    # Max 8% of portfolio vega per stock
    'sector_cap': 0.25,           # Max 25% in any sector
}

print('Strategy Parameters:')
for k, v in PARAMS.items():
    print(f'  {k}: {v}')

## Find Candidate Options

Filter options to find those matching our criteria.

In [ ]:
# Get options data for a test period
year, month = 2002, 4
options = db.query_month(year, month)
print(f'Total options: {len(options):,}')

In [ ]:
# Filter for strangle candidates
def find_strangle_candidates(df, dte_min=30, dte_max=60, delta_target=0.16, delta_tol=0.05):
    """
    Find put/call pairs suitable for short strangles.
    """
    df = df.copy()
    
    # Parse dates
    df['DataDate'] = pd.to_datetime(df['DataDate'])
    df['Expiration'] = pd.to_datetime(df['Expiration'])
    df['DTE'] = (df['Expiration'] - df['DataDate']).dt.days
    
    # Filter by DTE
    df = df[(df['DTE'] >= dte_min) & (df['DTE'] <= dte_max)]
    
    # Filter by delta (looking for OTM options near target delta)
    puts = df[(df['Type'] == 'put') & 
              (df['Delta'].abs() >= delta_target - delta_tol) &
              (df['Delta'].abs() <= delta_target + delta_tol)]
    
    calls = df[(df['Type'] == 'call') & 
               (df['Delta'].abs() >= delta_target - delta_tol) &
               (df['Delta'].abs() <= delta_target + delta_tol)]
    
    # Filter by liquidity (volume > 0 or open interest > 100)
    puts = puts[(puts['Volume'] > 0) | (puts['OpenInterest'] > 100)]
    calls = calls[(calls['Volume'] > 0) | (calls['OpenInterest'] > 100)]
    
    return puts, calls

puts, calls = find_strangle_candidates(
    options, 
    dte_min=PARAMS['dte_min'],
    dte_max=PARAMS['dte_max'],
    delta_target=PARAMS['delta_target']
)

print(f'Candidate puts: {len(puts):,}')
print(f'Candidate calls: {len(calls):,}')

In [ ]:
# Show sample candidates
print('Sample put candidates:')
puts[['UnderlyingSymbol', 'Strike', 'Expiration', 'Bid', 'Ask', 'Delta', 'IV', 'DTE']].head(10)

## Premium Analysis

Analyze the premium available for short strategies.

In [ ]:
# Calculate credit received for strangles by underlying
def calculate_strangle_credits(puts_df, calls_df):
    """
    Match puts and calls by underlying/date/expiration and calculate credits.
    """
    results = []
    
    # Group by underlying, date, expiration
    for (symbol, date, exp), put_group in puts_df.groupby(['UnderlyingSymbol', 'DataDate', 'Expiration']):
        call_group = calls_df[
            (calls_df['UnderlyingSymbol'] == symbol) &
            (calls_df['DataDate'] == date) &
            (calls_df['Expiration'] == exp)
        ]
        
        if len(put_group) > 0 and len(call_group) > 0:
            # Use the option closest to target delta
            best_put = put_group.iloc[(put_group['Delta'].abs() - 0.16).abs().argmin()]
            best_call = call_group.iloc[(call_group['Delta'].abs() - 0.16).abs().argmin()]
            
            # Calculate mid price credit
            put_mid = (best_put['Bid'] + best_put['Ask']) / 2
            call_mid = (best_call['Bid'] + best_call['Ask']) / 2
            total_credit = put_mid + call_mid
            
            results.append({
                'symbol': symbol,
                'date': date,
                'expiration': exp,
                'put_strike': best_put['Strike'],
                'call_strike': best_call['Strike'],
                'put_delta': best_put['Delta'],
                'call_delta': best_call['Delta'],
                'put_credit': put_mid,
                'call_credit': call_mid,
                'total_credit': total_credit,
                'underlying_price': best_put['UnderlyingPrice'],
                'avg_iv': (best_put['IV'] + best_call['IV']) / 2,
                'dte': best_put['DTE']
            })
    
    return pd.DataFrame(results)

strangles = calculate_strangle_credits(puts, calls)
print(f'Total strangle opportunities: {len(strangles):,}')
strangles.head(10)

In [ ]:
# Analyze credit as % of underlying price
if len(strangles) > 0:
    strangles['credit_pct'] = strangles['total_credit'] / strangles['underlying_price'] * 100
    
    print('Credit Statistics (% of underlying):')
    print(strangles['credit_pct'].describe())
    
    plt.figure(figsize=(10, 4))
    plt.hist(strangles['credit_pct'], bins=50, edgecolor='black')
    plt.xlabel('Credit (% of underlying)')
    plt.ylabel('Count')
    plt.title('Distribution of Strangle Credits')
    plt.tight_layout()
    plt.show()

## Next Steps

1. **Backtest framework**: Build P&L simulation with daily mark-to-market
2. **Beta hedging logic**: Implement rolling hedge adjustments
3. **Risk metrics**: Track vega, gamma, max drawdown
4. **Crash overlay**: Test convexity hedges